# Modelling with DL and word embedding transform

In [136]:
# try weighted word embedding
# https://machinelearningmastery.com/develop-word-embedding-model-predicting-movie-review-sentiment/

In [2]:
# https://towardsdatascience.com/multi-label-text-classification-with-scikit-learn-30714b7819c5
# https://docs.google.com/spreadsheets/d/1hg4KKKdivoh6mG7Bena0jYdJSvzgABlWLrU0NX_vHTQ/edit#gid=49385392

In [3]:
data_path='../../data/'
data_raw_path="../../data/raw/for second propose/"
data_man_path="../../data/manipulate/for second propose/"
model_path="../../model/modeling_second_propose/"

In [4]:
%matplotlib inline
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
# from nltk.corpus import stopwords
# stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn.linear_model import SGDClassifier
import seaborn as sns
import nltk
from string import digits
import ast
import itertools

In [5]:
stop_words=open(data_path+"stopwords_id.txt")
stop_words=set([i[:-1] for i in stop_words.readlines()])

# Get The Data

In [6]:
dtag=pd.read_csv(data_man_path+"data_tag_goshop2.csv").drop_duplicates()
# dtag.to_csv(data_man_path+"data_tag_goshop2.csv",index=False)

In [7]:
dtext2=pd.read_csv(data_man_path+"data_raw_path_w_label2.csv")
dtext2["real_tags"]=dtext2.real_tags.apply(lambda x : set(ast.literal_eval(x)))

In [8]:
dtext2["num_tags"]=dtext2.real_tags.apply(lambda x: len(x))

In [9]:
dtag["num_word"]=dtag.Keywords.apply(lambda x : len(x.split(" ")))
dtag["num_word_tag"]=dtag.Tags.apply(lambda x : len(x.split(" ")))

In [10]:
keyword_set=set(dtag.Keywords.unique())
keyword_set1=set([t for t in keyword_set if len(t.split(" "))==1])
keyword_set2=set([t for t in keyword_set if len(t.split(" "))==2])
keyword_set3=set([t for t in keyword_set if len(t.split(" "))==3])


ts=set(dtag.Tags.unique())
ts1=set([t for t in ts if len(t.split(" "))==1])
ts2=set([t for t in ts if len(t.split(" "))==2])
ts3=set([t for t in ts if len(t.split(" "))==3])

In [11]:
dict_keyword=pd.DataFrame(dtag.Tags)
dict_keyword.index=dtag.Keywords
dict_keyword=dict_keyword["Tags"].to_dict()

dict_sc=pd.DataFrame(dtag["Sub-Categories"])
dict_sc.index=dtag.Tags
dict_sc=dict_sc["Sub-Categories"].to_dict()

In [12]:
print(len(ts1),len(ts2))

267 166


In [13]:
dtext2.sample(100).head(20)

,booking_time,order_no,voucher_id,item_list,payment_method_name,item_list_cln1,item_list_cln2,real_tags,num_tags
732190,2017-01-20 06:42:40,280920986,NaN,sticker atas nama febby,CASH,sticker atas nama febby,sticker atas nama febby,{},0
1429766,2018-06-04 03:24:05,478234377,NaN,royal canin warna pink buat kitten yang 2kg,CASH,royal canin warna pink buat kitten yang kg,royal canin warna pink buat kitten yang kg,{royal canin},1
1053639,2018-05-23 18:10:14,477597997,NaN,"2 rawon, 1 telor asin",CASH,"rawon, telor asin",rawon telor asin,"{rawon, telur asin}",2
384469,2017-08-27 04:47:31,355710725,NaN,nasi rendang + ayam 1nasi rendang 1nasi ayam b...,CASH,"nasi rendang , ayam nasi rendang nasi ayam bal...",nasi rendang ayam nasi rendang nasi ayam balad...,"{ayam gulai, rendang, nasi}",3
300127,2018-03-29 04:45:18,464798771,NaN,klengkeng 1kgjeruk 1kg,CASH,klengkeng kgjeruk kg,klengkeng kgjeruk kg,{jeruk},1
469699,2017-04-10 04:05:07,300953425,NaN,5x es kopi susu tetangga,CASH,x es kopi susu tetangga,x es kopi susu tetangga,{kopi susu},1
1231550,2016-06-04 10:37:17,151121034,NaN,makaroni kering pedes 3 bungkus,CASH,makaroni kering pedes bungkus,makaroni kering pedes bungkus,{makaroni},1
180356,2018-03-13 05:28:36,456142902,NaN,Obat Yuricky,CASH,obat yuricky,obat yuricky,{obat},1
1410095,2017-09-30 07:59:36,369993772,NaN,siomay 3 pack nampan sdh ready,CASH,siomay pack nampan sdh ready,siomay pack nampan sdh ready,{siomay},1
1453217,2018-08-02 02:40:36,481836227,NaN,soto,CASH,soto,soto,{soto},1


In [14]:
len(dtext2)

1499625

### Modeling

In [141]:
import keras.backend as K

In [142]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float(len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)


def custom_loss_func2(model,X,y_true0):
    y_pred0=model.predict(X)
    return hamming_score(y_true0, y_pred0)


def print_score(y_pred, clf,y_test_tfidf):
    print("Clf: ", clf)
    print("Hamming loss: {}".format(hamming_loss(y_test_tfidf,y_pred)))
    try:
        print("Hamming score: {}".format(hamming_score(y_test_tfidf,y_pred)))
    except:
        print("Hamming score: {}".format(hamming_score(y_test_tfidf,y_pred.toarray())))
        
    print("---")    
    
def check_tag2(list_X,i):
    if i in list_X:
        return 1
    else :
        return 0
    

# https://stats.stackexchange.com/questions/207794/what-loss-function-for-multi-class-multi-label-classification-tasks-in-neural-n    
def abs_KL_div(y_true, y_pred):
    y_true = K.clip(y_true, K.epsilon(), None)
    y_pred = K.clip(y_pred, K.epsilon(), None)
    return K.sum( K.abs( (y_true- y_pred) * (K.log(y_true / y_pred))), axis=-1)

In [113]:
from sklearn.preprocessing import MultiLabelBinarizer
from imblearn.over_sampling import RandomOverSampler,SMOTE
from sklearn.metrics import hamming_loss
from sklearn.preprocessing import LabelEncoder
from skmultilearn.problem_transform import LabelPowerset,ClassifierChain
from skmultilearn.adapt import MLkNN
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from gensim.models import Word2Vec
from gensim.models.wrappers import FastText
from gensim.models import KeyedVectors
from collections import defaultdict
from sklearn.preprocessing import RobustScaler

In [114]:
w2v_path="../../data/w2v/"

In [115]:
# embd2=Word2Vec.load(w2v_path+'our_vocab.bin')
embd2=Word2Vec.load(w2v_path+'our_vocab2.bin')

In [116]:
len(embd2["buku"])

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


95

In [117]:
embd2.layer1_size

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `layer1_size` (Attribute will be removed in 4.0.0, use self.trainables.layer1_size instead).
  if __name__ == '__main__':


95

In [118]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [119]:
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, embedding.layer1_size))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        try:
            weight_matrix[i] = embedding[word]
        except:
            weight_matrix[i] = np.zeros(embedding.layer1_size)
    return weight_matrix

In [120]:
## remove zero tags
dtext3=dtext2[dtext2.num_tags!=0]
dtext3.head()

,booking_time,order_no,voucher_id,item_list,payment_method_name,item_list_cln1,item_list_cln2,real_tags,num_tags
0,2017-06-23 14:05:01,330184060,NaN,caramel coklat,CASH,caramel coklat,caramel coklat,{cokelat},1
1,2018-08-08 10:44:44,482277109,NaN,Sate Ayam (daging aja) 40 tusuk,CASH,sate ayam daging aja tusuk,sate ayam daging aja tusuk,"{daging, sate ayam}",2
3,2018-04-14 08:03:46,471187625,NaN,krimcis duren 1,GO-PAY,krimcis duren,krimcis duren,{durian},1
4,2018-09-05 07:45:29,484118692,NaN,pecel lele 1 ( pakai cabe ayam rica - rica) t...,CASH,"pecel lele pakai cabe ayam rica , rica tidak p...",pecel lele pakai cabe ayam rica rica tidak pak...,"{ayam, pecel lele, nasi}",3
5,2018-08-17 09:08:30,482854178,NaN,durian /box,CASH,durian box,durian box,{durian},1


In [121]:
len(dtext3)

1091020

In [122]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(dtext3.real_tags)

MultiLabelBinarizer(classes=None, sparse_output=False)

In [123]:
# multilabel_binarizer.classes_

In [124]:
train2, test2 = train_test_split(dtext3.sample(1000000,random_state=231), random_state=42, test_size=0.2, shuffle=True)

X_train2=train2.item_list_cln2
X_test2=test2.item_list_cln2

Y_train2=multilabel_binarizer.transform(train2.real_tags)
Y_test2=multilabel_binarizer.transform(test2.real_tags)

In [125]:
# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(X_train2)

In [126]:
## Training
# sequence encode
encoded_docs = tokenizer.texts_to_sequences(X_train2)
# pad sequences
max_length = max([len(s.split()) for s in X_train2])
Xtr2 = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [127]:
Xtr2.shape

(800000, 82)

In [128]:
## Testing
# sequence encode
encoded_docs = tokenizer.texts_to_sequences(X_test2)
# pad sequences
# max_length = max([len(s.split()) for s in X_test2])
Xte2 = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [129]:
vocab_size = len(tokenizer.word_index) + 1

In [130]:
embedding_vectors = get_weight_matrix(embd2, tokenizer.word_index)

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: Call to deprecated `layer1_size` (Attribute will be removed in 4.0.0, use self.trainables.layer1_size instead).
/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:11: DeprecationWarning: Call to deprecated `layer1_size` (Attribute will be removed in 4.0.0, use self.trainables.layer1_size instead).


In [131]:
embedding_layer = Embedding(vocab_size, embd2.layer1_size, weights=[embedding_vectors], 
                            input_length=max_length, trainable=False)

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `layer1_size` (Attribute will be removed in 4.0.0, use self.trainables.layer1_size instead).
  if __name__ == '__main__':


In [132]:
Y_train2.shape[1]

448

In [143]:
# define model
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(800, activation='relu'))
model.add(Dense(800, activation='relu'))
model.add(Dense(Y_train2.shape[1], activation='sigmoid'))
print(model.summary())
# compile network
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss=abs_KL_div, optimizer='adam', metrics=['accuracy'])



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 82, 95)            14422805  
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 78, 64)            30464     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 39, 64)            0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 2496)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 800)               1997600   
_________________________________________________________________
dense_18 (Dense)             (None, 800)               640800    
_________________________________________________________________
dense_19 (Dense)             (None, 448)               358848    
Total para

In [ ]:
# fit network
model.fit(Xtr2, Y_train2, epochs=3, batch_size=700, verbose=1)

Epoch 1/3
253400/800000 [========>.....................] - ETA: 9:16 - loss: 60.9724 - acc: 0.0065918   

In [ ]:

# evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

In [ ]:
y_pred = clf.predict(X_te2)
print_score(y_pred, clf, Y_test2)

In [43]:
## DecTree
clf2=joblib.load(model_path+"dectree_second_tags_2_680.pkl")
y_pred2=clf2.predict(X_te2)
test2["tag_pred"]=multilabel_binarizer.inverse_transform(y_pred2)

In [44]:
test2.sample(300).head(20)

,booking_time,order_no,voucher_id,item_list,payment_method_name,item_list_cln1,item_list_cln2,real_tags,num_tags,tag_pred
629657,2018-05-18 08:47:58,477310088,NaN,vitamin wajah jelly blue,CASH,vitamin wajah jelly blue,vitamin wajah jelly blue,{vitamin},1,"(vitamin,)"
321247,2017-06-07 13:02:02,321835718,NaN,1/2 loyang kue bandung coklat keju (sedikit me...,PARTIAL_PAYMENT,loyang kue bandung coklat keju sedikit mentega,loyang kue bandung coklat keju sedikit mentega,"{cokelat, kue}",2,"(cokelat,)"
982267,2017-10-29 02:50:29,383807610,NaN,cakue goreng biasa 5 biji,CASH,cakue goreng biasa biji,cakue goreng biasa biji,{cakue},1,"(cakue,)"
288352,2016-08-02 11:15:06,179576511,NaN,"Beefy cheesy chicken : 2. (1 kentang, 1 nasi)",CASH,"beefy cheesy chicken , , kentang, nasi",beefy cheesy chicken kentang nasi,"{nasi, kentang, ayam}",3,"(ayam,)"
1206355,2017-11-06 04:07:58,388494134,NaN,Rujak samping gramedia. Cabenya 4,CASH,"rujak samping gramedia, cabenya",rujak samping gramedia cabenya,{rujak},1,"(buah, rujak)"
1274322,2018-06-07 14:40:04,478472122,NaN,beli rokok evolution hijau menthol,CASH,beli rokok evolution hijau menthol,beli rokok evolution hijau menthol,{rokok},1,"(rokok,)"
218280,2017-12-18 04:51:33,411272923,NaN,nasi putih 4 porsisiobak 4 porsisate plecing 2...,CASH,nasi putih porsisiobak porsisate plecing porsi,nasi putih porsisiobak porsisate plecing porsi,"{nasi putih, sate}",2,"(nasi putih,)"
1264719,2018-05-15 16:56:44,477173654,NaN,ketoprak 5,CASH,ketoprak,ketoprak,{ketoprak},1,"(ketoprak,)"
1241417,2018-05-25 06:45:46,477671411,NaN,kuah bakut sayur asin 1 porsitelor pete 1 porsi,CASH,kuah bakut sayur asin porsitelor pete porsi,kuah bakut sayur asin porsitelor pete porsi,"{pete, telur, sayur}",3,"(ayam, kerupuk, lontong sayur, nasi, telur asi..."
1025513,2018-05-13 16:51:16,477057552,NaN,2 bungkus djarum super8 aqua dingin ukuran sedang,CASH,bungkus djarum super aqua dingin ukuran sedang,bungkus djarum super aqua dingin ukuran sedang,{aqua},1,"(rokok,)"


In [47]:
test1=test2[["item_list_cln2","real_tags","tag_pred"]]

In [51]:
for c in ts:
    cek_tag=test1.real_tags.apply(lambda x : check_tag2(x,c))
    test1[c]=cek_tag

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [54]:
acc=[]
benc=[]
lrate=[]
auc_list=[]
for category in list(ts):
    # compute the testing accuracy
    True_tag=test2.real_tags.apply(lambda x: check_tag2(x,category)).as_matrix()
    Pred_tag=test2.tag_pred.apply(lambda x: check_tag2(x,category)).as_matrix()
    
    acc_score=accuracy_score(True_tag,Pred_tag)
    try:
        auc_cat=roc_auc_score(True_tag,Pred_tag)
    except:
        auc_cat=0.001
    
    print('Test accuracy is {}'.format(acc_score))
    if True_tag.mean()<0.5:
        benchm=1-test1[category].mean()
    else:
        benchm=test1[category].mean()
    print('diff with benchmark {}'.format(acc_score-benchm))
    
    acc.append(acc_score)
    benc.append(benchm)
    lrate.append(((acc_score-benchm)/float(benchm))*100)
    auc_list.append(auc_cat)
    
dFRes2=pd.DataFrame({"tags":list(ts),"accuracy":acc,"benchmark":benc,"AUC":auc_list,"learn_rate(%)":lrate})

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Test accuracy is 0.997
diff with benchmark 0.0031250000000000444
Test accuracy is 0.9968455882352941
diff with benchmark -0.0004191176470588198
Test accuracy is 0.9967573529411765
diff with benchmark 0.002742647058823544
Test accuracy is 0.9924926470588236
diff with benchmark 0.0006764705882352784
Test accuracy is 0.9767132352941177
diff with benchmark 0.0021838235294118213
Test accuracy is 1.0
diff with benchmark 0.0
Test accuracy is 0.9981985294117647
diff with benchmark 0.0014705882352941124
Test accuracy is 0.9986544117647059
diff with benchmark -0.00023529411764700026
Test accuracy is 0.9993308823529412
diff with benchmark -0.00016911764705884735
Test accuracy is 0.9996911764705882
diff with benchmark 4.411764705880561e-05
Test accuracy is 0.9961838235294118
diff with benchmark 0.000786764705882459
Test accuracy is 0.9999558823529412
diff with benchmark -7.352941176486105e-06
Test accuracy is 0.9994705882352941
diff with benchmark 0.00016176470588236125
Test accuracy is 0.99980882

In [55]:
dFRes2["learn_rate(%)"].describe()

count    451.000000
mean       0.090442
std        0.231875
min       -0.395313
25%       -0.002216
50%        0.020627
75%        0.100687
max        1.960055
Name: learn_rate(%), dtype: float64

In [56]:
dFRes2["accuracy"].describe()

count    451.000000
mean       0.996851
std        0.006440
min        0.926338
25%        0.997018
50%        0.998471
75%        0.999353
max        1.000000
Name: accuracy, dtype: float64

In [57]:
dFRes2["AUC"].describe()

count    451.000000
mean       0.707284
std        0.123865
min        0.001000
25%        0.648067
50%        0.716729
75%        0.784646
max        0.935547
Name: AUC, dtype: float64

In [58]:
dFRes2.sample(100).head(15)

,tags,accuracy,benchmark,AUC,learn_rate(%)
92,bintang,0.997941,0.997221,0.750984,0.072260
337,ketoprak,0.998272,0.996515,0.844612,0.176350
34,ayam pop,0.998956,0.998838,0.737125,0.011778
175,roti bakar,0.998382,0.997728,0.766710,0.065590
290,soto ayam,0.996581,0.996397,0.698352,0.018449
329,tremenza,0.999632,0.999654,0.680778,-0.002207
215,ikan bakar,0.997346,0.997772,0.647970,-0.042742
12,kemeja,0.999471,0.999309,0.744593,0.016188
231,royal canin,0.999110,0.996257,0.928114,0.286366
334,degirol,0.999824,0.999853,0.549978,-0.002942


In [60]:
# ', '.join(dFRes2[dFRes2.AUC<=0.8].tags.tolist())

In [62]:
(dFRes2.AUC>0.8).mean()*415

85.57649667405765

In [63]:
# test1[test1["rujak kuah"]==1][["item_list_cln2","real_tags"]]

In [81]:
# dFRes2[dFRes2.AUC<0.8].tags.tolist()